In [32]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pynbody as pb
import os

from scipy import integrate

from astropy import units as u
from astropy.constants import G
simT = u.year/(2*np.pi)
simV = u.AU / simT

import sys
sys.path.insert(0, '/astro/users/scw7/OrbitTools/')
import OrbitTools

from profileTools import *

In [28]:
m_central = 0.13
m_central_cgs = (m_central*u.M_sun).to(u.g).value
T_in, T_out = 1, 100 # days

r_in, r_out = sma((T_in*u.d).to(u.s).value, m_central_cgs), \
              sma((T_out*u.d).to(u.s).value, m_central_cgs)
r = np.linspace(r_in, r_out)
r_AU = (r*u.cm).to(u.AU).value

# Hayashi 1981
alpha = 3/2
A = 6

f = m_central

surf_den_pl = sigma_pl(alpha, A, r_AU, f)

yvals = surf_den_pl*r
m_disk = 2*np.pi*integrate.simps(yvals, r)
m_disk_earth = (m_disk*u.g).to(u.M_earth).value
print('Total disk mass is ' + str(m_disk_earth) + ' Earth masses')

s_pl = (100*u.km).to(u.cm).value
rho_pl = 2
m_pl = 4/3*np.pi*s_pl**3

N_pl = int(m_disk/m_pl)
print(str(N_pl) + ' planetesimals required')
ntotal = N_pl + 1

Total disk mass is 0.13320544623251612 Earth masses
189923 planetesimals required


In [36]:
# Draw planetesimal semimajor axes from a probability distribution that follows the
# surface density profile

N_pl = 10000

def p(A, x):
    return A*x**(-3/2 + 1)

def cum_p(A, x, xmin):
    x_vals = np.linspace(xmin, x)
    return integrate.simps(p(A, x_vals), x_vals)

xmin, xmax = r_in, r_out
xvals = np.linspace(xmin, xmax)
A = 1/(integrate.simps(p(1, xvals), xvals))

uniform_x = np.random.rand(N_pl)*(r_out - r_in) + r_in
a_random = np.empty(N_pl)
for idx in range(N_pl):
    a_random[idx] = (1 - cum_p(A, uniform_x[idx], xmin))*(r_out - r_in) + r_in

In [37]:
# Eccentricities and inclinations come from a rayleigh distribution with inclination
# dist half as large as ecc dist
# Mode of distribution chosen so that stopping time from gas and relaxation time are equal

omega = np.sqrt(G.cgs.value*m_central_cgs/r**3)
C_D = 1
vk = np.sqrt(G.cgs.value*m_central_cgs/r)

# Values for M stars from Backus + Quinn 2016
q = 0.59
T0 = 130
mu = 2.34 # Hayashi 1981
gas_to_dust = 0.01

cs = soundspeed(T0, mu, q, r_AU)
vgas = v_gas(m_central, r, cs, q)
rhogas = rho_gas(cs, omega, surf_den_pl, gas_to_dust)
e_std = e_eq(omega, surf_den_pl, m_pl, s_pl, C_D, vk, rhogas, vgas)

# Bin particles by semimajor axis and assign eccentricites and inclinations based on above profile
# Orbital phase and orientation assigned randomly
a_vals = np.sort(a_random)
ecc_vals = np.empty(len(a_sorted))
inc_vals = np.empty(len(a_sorted))
omega_vals = np.empty(len(a_sorted))
Omega_vals = np.empty(len(a_sorted))
M_vals = np.empty(len(a_sorted))

e_half = 0.5*(e_std[1:] + e_std[:-1])
p_idx = 0
for idx in range(len(r)-1):
    mask = (a_vals >= r[idx]) & (a_vals < r[idx+1])
    a_within = a_vals[mask]
    e_std = e_half[idx]
    i_std = e_std/2
    n_within = len(a_within)
    ecc_vals[p_idx:p_idx+n_within] = np.random.rayleigh(e_std, n_within)
    inc_vals[p_idx:p_idx+n_within] = np.random.rayleigh(e_std, n_within)
    omega_vals[p_idx:p_idx+n_within] = np.random.rand(n_within)*2*np.pi
    Omega_vals[p_idx:p_idx+n_within] = np.random.rand(n_within)*2*np.pi
    M_vals[p_idx:p_idx+n_within] = np.random.rand(n_within)*2*np.pi
    p_idx += n_within

In [38]:
filename = 'stip.ic'
time = 0

masses = np.empty(ntotal)
positions = np.empty((ntotal, 3))
velocities = np.empty((ntotal, 3))
eps = np.empty(ntotal)

for idx in range(N_pl):
    pos, vel = OrbitTools.kep2cart(a_vals[idx], ecc_vals[idx], inc_vals[idx],\
                                   Omega_vals[idx], omega_vals[idx], M_vals[idx], masses[idx], m_central)
    positions[idx+1] = pos
    velocities[idx+1] = vel
    masses[idx+1] = (m_pl*u.g).to(u.M_sun).value
    eps[idx+1] = (s_pl*u.cm).to(u.AU).value/2
    
# Gravitational potential, not used for anything
pot = np.zeros(ntotal)

In [39]:
# Set the position and velocity of the central star

masses[0] = m_central
eps[0] = 1e-10

m_tot = np.sum(masses)
r_com_x = np.sum(positions[:,0][1:]*masses[1:])/m_tot
r_com_y = np.sum(positions[:,1][1:]*masses[1:])/m_tot
r_com_z = np.sum(positions[:,2][1:]*masses[1:])/m_tot

v_com_x = np.sum(velocities[:,0][1:]*masses[1:])/m_tot
v_com_y = np.sum(velocities[:,1][1:]*masses[1:])/m_tot
v_com_z = np.sum(velocities[:,2][1:]*masses[1:])/m_tot

positions[0] = -r_com_x, -r_com_y, -r_com_z
velocities[0] = -v_com_x, -v_com_y, -v_com_z

In [40]:
f = open('ic.txt', 'w')

f.write(str(ntotal) + ', 0, 0\n')
f.write(str(3) + '\n')
f.write(str(time) + '\n')

for idx in range(ntotal):
    f.write(str(masses[idx]) + '\n')

for idx in range(ntotal):
    f.write(str(positions[:,0][idx]) + '\n')
for idx in range(ntotal):
    f.write(str(positions[:,1][idx]) + '\n')
for idx in range(ntotal):
    f.write(str(positions[:,2][idx]) + '\n')
    
for idx in range(ntotal):
    f.write(str(velocities[:,0][idx]) + '\n')
for idx in range(ntotal):
    f.write(str(velocities[:,1][idx]) + '\n')
for idx in range(ntotal):
    f.write(str(velocities[:,2][idx]) + '\n')
    
for idx in range(ntotal):
    f.write(str(eps[idx]) + '\n')
    
for idx in range(ntotal):
    f.write(str(pot[idx]) + '\n')

f.close()

In [41]:
os.system("$HOME/tipsy_tools/ascii2bin < ic.txt > " + filename)
os.system("rm ic.txt")

0

In [68]:
# Recommended orbital timestep
def t_step(sma):
    return np.sqrt(sma**3/(G.cgs.value*m_central_cgs))*0.03

t_step_in, t_step_out = t_step(r_in), t_step(r_out)
print('Recommended inner timestep: ' + str((t_step_in*u.s).to(simT).value/(2*np.pi)))
print('Recommended outer timestep: ' + str((t_step_out*u.s).to(simT).value/(2*np.pi)))
print('(in simulation units)')
rung = np.log2(t_step_out/t_step_in)
print('If stepping at outer timestep, inner bodies will be on rung ' + str(rung))

Recommended inner timestep: 1.3072274586603282e-05
Recommended outer timestep: 0.001307227458660328
(in simulation units)
If stepping at outer timestep, inner bodies will be on rung 6.643856189774724
